In [17]:
import torch
import pickle
def load_inputs_from_file(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

# 从文件中读取数据
loaded_inputs1 = load_inputs_from_file('inputs.pkl')
loaded_inputs2 = load_inputs_from_file('true.pkl')


In [18]:
print(loaded_inputs1['input_ids'][0])
print(loaded_inputs2['input_ids'][0])

tensor([    1,     0,     0,  ...,  1009,  4423, 29973], device='cuda:0')
tensor([    1,     0,     0,  ...,  1009,  4423, 29973], device='cuda:0')


In [ ]:
from transformers import AutoModelForCausalLM, LlamaTokenizer
TOKENIZER_PATH = "/mnt/pfs-mc0p4k/nlu/team/yuhaofu/modle_weight/vicuna-7b-v1.5"
tokenizer = LlamaTokenizer.from_pretrained(TOKENIZER_PATH)
id = tokenizer.encode([' '])
print(id)


/opt/conda/envs/cogvlm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/cogvlm/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


[1, 0]


In [11]:
response1 = tokenizer.decode(loaded_inputs1['input_ids'][0])
response2 = tokenizer.decode(loaded_inputs2['input_ids'][0])
print(response1)
print(response2)

<s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><u

In [16]:
response = tokenizer.decode('29871')
print(response)

</s><s>


In [25]:
import re
def parse_bbox_from_response(response):
    """
    从模型响应中解析边界框坐标
    """
    # 寻找[x, y, x2, y2]格式的坐标，包括整数和小数
    pattern = r'\[\[(\d+),\s*(\d+),\s*(\d+),\s*(\d+)\]\]'
    match = re.search(pattern, response)
    
    if match:
        try:
            x1 = int(match.group(1))  # 提取整数值
            y1 = int(match.group(2))
            x2 = int(match.group(3))
            y2 = int(match.group(4))
            # 验证坐标有效性，如果仍需要确保有效性范围
            if x1 <= x2 and y1 <= y2:
                x1 = x1/1000
                x2 = x2/1000
                y1 = y1/1000
                y2 = y2/1000
                return [x1, y1, x2, y2]
        except ValueError:
            pass
    
    # 不符合条件的返回None
    return None

In [26]:
response = "[[270,159,624,975]]"
bbox = parse_bbox_from_response(response)
print(bbox)

[0.27, 0.159, 0.624, 0.975]


In [4]:
import json
file1 = '/mnt/pfs-mc0p4k/nlu/team/yuhaofu/data/LLaVA-CoT-100k/fgeo_1_10_bbox.json'
file2 = '/mnt/pfs-mc0p4k/nlu/team/yuhaofu/data/LLaVA-CoT-100k/filter_geoqa+_1_10.json'
# 读取两个 JSON 文件
with open(file1, 'r') as f:
    data1 = json.load(f)

with open(file2, 'r') as f:
    data2 = json.load(f)

# 存储不满足条件的样本 ID
mismatched_ids = []

# 遍历第一个 JSON 文件中的每个记录
for item1 in data1:
    id1 = item1['id']
    
    # 在第二个 JSON 文件中找到匹配的记录
    item2 = next((item for item in data2 if item["id"] == id1), None)
    if item2 is None:
        continue

    # 检查第二个文件的 conversations 长度是否为第一个文件的两倍
    if len(item2['conversations']) != 2 * len(item1['conversations']):
        mismatched_ids.append(id1)

# 打印不满足条件的样本 ID
print("不满足条件的样本ID：", mismatched_ids)

不满足条件的样本ID： []


In [6]:
import json
import re

# 读取两个 JSON 文件
with open(file1, 'r') as f:
    data1 = json.load(f)

with open(file2, 'r') as f:
    data2 = json.load(f)

# 遍历第一个 JSON 文件中的每个记录
for item1 in data1:
    id1 = item1['id']
    
    # 在第二个 JSON 文件中找到匹配的记录
    item2 = next((item for item in data2 if item["id"] == id1), None)
    if item2 is None:
        continue

    # 遍历第一个记录中的 conversations，同时与第二个文件的对话对应
    for idx, conversation1 in enumerate(item1['conversations']):
        if idx * 2 + 1 >= len(item2['conversations']):
            break
        
        # 获取对应的 from gpt 文本
        gpt_response = item2['conversations'][idx * 2 + 1]['value']
        
        # 处理 <CAPTION>
        if "<CAPTION>" in conversation1 and conversation1["<CAPTION>"] is not None:
            caption_phrases = {phrase["content"]: phrase["bbox"] for phrase in conversation1["<CAPTION>"] if phrase["bbox"] is not None}
            caption_content = re.search(r'<CAPTION>(.*?)</CAPTION>', gpt_response, re.DOTALL)
            if caption_content:
                updated_caption = caption_content.group(1)
                for phrase, bbox in caption_phrases.items():
                    if phrase in updated_caption:
                        updated_caption = updated_caption.replace(phrase, f"{phrase} {bbox}")
                gpt_response = gpt_response.replace(caption_content.group(1), updated_caption)
        
        # 处理 <REASONING>
        if "<REASONING>" in conversation1 and conversation1["<REASONING>"] is not None:
            reasoning_phrases = {phrase["content"]: phrase["bbox"] for phrase in conversation1["<REASONING>"] if phrase["bbox"] is not None}
            reasoning_content = re.search(r'<REASONING>(.*?)</REASONING>', gpt_response, re.DOTALL)
            if reasoning_content:
                updated_reasoning = reasoning_content.group(1)
                for phrase, bbox in reasoning_phrases.items():
                    if phrase in updated_reasoning:
                        updated_reasoning = updated_reasoning.replace(phrase, f"{phrase} {bbox}")
                gpt_response = gpt_response.replace(reasoning_content.group(1), updated_reasoning)

        # 更新 gpt_response
        item2['conversations'][idx * 2 + 1]['value'] = gpt_response

# 输出处理后的 JSON 数据
with open('/mnt/pfs-mc0p4k/nlu/team/yuhaofu/data/LLaVA-CoT-100k/test.json', 'w') as f:
    json.dump(data2, f, indent=2)